In [7]:
!pip install transformers torch
import pprint


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier([
    "I've been waiting for a long time, and I'm really excited about this new feature!", 
    "I don't want to do this anymore",
])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9998000264167786},
 {'label': 'NEGATIVE', 'score': 0.9983810186386108}]

In [5]:
# Getting the tokenizer from the model
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [16]:
raw_inputs = [
    "I've been waiting for a long time, and I'm really excited about this new feature!",
    "I don't want to do this anymore",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
pprint.pp(inputs["input_ids"][1].size())

torch.Size([23])


In [ ]:
"""
 This retrieves the checkpoint and loads the model.
 It only loads the base model, which outputs hidden states, also known as features. 
"""
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [27]:
outputs = model(**inputs)
pprint.pp(outputs.last_hidden_state.shape)
pprint.pp(outputs)

torch.Size([2, 23, 768])
BaseModelOutput(last_hidden_state=tensor([[[ 0.6624,  0.1321,  0.3554,  ...,  0.3924,  0.8493, -0.5336],
         [ 1.1194,  0.4005,  0.1795,  ...,  0.2101,  1.0571, -0.0928],
         [ 1.4226,  0.1522,  0.4107,  ...,  0.7130,  0.1705, -0.8750],
         ...,
         [ 0.7327,  0.2963,  0.3484,  ...,  0.4382,  0.5833, -0.3171],
         [ 0.7167,  0.1841,  0.1809,  ...,  0.3667,  0.7874, -0.3829],
         [ 0.8304,  0.1011,  0.4583,  ...,  0.4955,  0.7535, -0.6079]],

        [[-0.2206,  0.6148,  0.4590,  ...,  0.2267, -0.7586,  0.2361],
         [-0.1971,  0.7545,  0.3374,  ...,  0.0924, -0.5861,  0.1437],
         [-0.4712,  0.8037,  0.6487,  ...,  0.0371, -0.7031, -0.0427],
         ...,
         [-0.1700,  0.5407,  0.0977,  ...,  0.1877, -0.6209, -0.0272],
         [-0.2191,  0.5828,  0.2055,  ...,  0.0163, -0.6439, -0.1589],
         [-0.2642,  0.3918,  0.3973,  ..., -0.2437, -0.6255, -0.0780]]],
       grad_fn=<NativeLayerNormBackward0>),
             

In [31]:
"""
 In order to get an output that is interpretable, we need to get a model with a sequence classification head.
 In this case, the AutoModelForSequenceClassification class is used.
"""

from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
pprint.pp(outputs.logits.shape)
pprint.pp(outputs)

torch.Size([2, 2])
SequenceClassifierOutput(loss=None,
                         logits=tensor([[-4.1100,  4.4068],
        [ 3.4950, -2.9293]], grad_fn=<AddmmBackward0>),
                         hidden_states=None,
                         attentions=None)


In [32]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[2.0005e-04, 9.9980e-01],
        [9.9838e-01, 1.6190e-03]], grad_fn=<SoftmaxBackward0>)


In [33]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}